In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.utils import *
from delta import *
from datetime import *
import hashlib
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  

In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("NB_Fact") 
# Spark will automatically use the master specified in spark-defaults.conf
spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [177]:
trgt_path_processed = "/mnt/Fact/Fact_Parquet"
trgt_path_csv = "/mnt/Fact/Fact_Processed"
mount_d ="/data/"
trgt_path = '/mnt/'
source_path = mount_d+"Template.xlsm"
sheet_name ='SPENDING_HISTORY'
calendar_path ="/mnt/Calendar/Calendar_Parquet/"


In [218]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    max_date = spark.sql(f"select max(date) from delta.`{calendar_path}` WHERE datesk = (select max(datesk) from delta.`{trgt_path_processed}`)").collect()[0][0] 
else:
    max_date= spark.sql(f"select min(date) from delta.`{calendar_path}`").collect()[0][0]
max_date= spark.sql(f"select min(date) from delta.`{calendar_path}`").collect()[0][0]
print(type(max_date))
print(max_date)

<class 'datetime.date'>
2000-01-01


In [219]:
pandas_df = pd.read_excel(source_path, sheet_name = sheet_name)
df=spark.createDataFrame(pandas_df)
df= df.withColumn("Date", date_format(df["Date"], "yyyy-MM-dd").cast("date"))

In [220]:
df.createOrReplaceTempView("vw_src")
df_src = spark.sql(f"select * from vw_src where Date >= '{max_date}'")
df_src.show()
print(df_src.count())


+-----+----------+--------------+--------------------+---------------+------------+
|Index|      Date| Spending Item|         Wallet used|       Category|Spend Amount|
+-----+----------+--------------+--------------------+---------------+------------+
|    1|2024-10-01|      Bus Fare|           ICICI Pay|     Travelling|         271|
|    2|2024-10-01|   Zomato Food|       Amazon Wallet|           Food|         436|
|    3|2024-10-01|Phone Recharge|           ICICI Pay|Mobile Recharge|         359|
|    4|2024-10-01|Phone Recharge|Credit Card - Amazon|Mobile Recharge|         118|
|    5|2024-10-01|  food from FC|               G-pay|           Food|         120|
|    6|2024-10-01|   Zomato Food|       Amazon Wallet|           Food|         160|
|    7|2024-10-01|     Groceries|           ICICI Pay|           Food|         245|
|    8|2024-10-02|         Iwish|           ICICI Pay|     Investment|         745|
|    9|2024-10-02|      TVM Rent|               G-pay|           Rent|      

In [221]:
df_temp =(
df_src
    .withColumnRenamed("Spending Item", "SpendingItem")
    .withColumnRenamed("Spend Amount", "SpendAmount")
    .withColumnRenamed("Wallet used", "WalletUsed")
)

In [222]:

df_output =(
    df_temp
    .withColumn("WalletSK", xxhash64("WalletUsed"))
    .withColumn("categorysk", xxhash64("category").cast("long"))
    .withColumn("DateSK", regexp_replace("date", "-", "").cast("string"))
    .withColumn("PKSK", xxhash64(concat("category", "WalletUsed", "Index", "Date")).cast("string"))
    .withColumn("UpdateTimeStamp", date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))
    .withColumn("RowSK", xxhash64(concat_ws("|", *[col(c) for c in df_temp.columns])))
    .drop("Index", "Date", "category","WalletUsed")
)

In [223]:
df_output.show()

+--------------+-----------+--------------------+--------------------+--------+--------------------+-------------------+--------------------+
|  SpendingItem|SpendAmount|            WalletSK|          categorysk|  DateSK|                PKSK|    UpdateTimeStamp|               RowSK|
+--------------+-----------+--------------------+--------------------+--------+--------------------+-------------------+--------------------+
|      Bus Fare|        271| 6156857810389859050|-6960913322501014326|20241001|-6565386604912707528|2024-11-21 16:04:44| 6693242557264883629|
|   Zomato Food|        436|-2996829708924805941|-1486987501743003025|20241001| 5938848620272978843|2024-11-21 16:04:44| 4075940645094743113|
|Phone Recharge|        359| 6156857810389859050|-6072606072650303353|20241001| 4210959322690640852|2024-11-21 16:04:44|-8240864996273807886|
|Phone Recharge|        118| 4962050552325147656|-6072606072650303353|20241001|  813826478454288165|2024-11-21 16:04:44|-7125262503767821305|
|  foo

In [224]:
df_output.createOrReplaceTempView("vw_source")

duplicate_counts = spark.sql("""
    SELECT COUNT(PKSK) as count
    FROM vw_source
    GROUP BY PKSK
    HAVING COUNT(PKSK) > 1
""")
x = [row['count'] for row in duplicate_counts.collect()]

# Fail the code if there are duplicates
if len(x) > 0:
    raise ValueError(f"Duplicate values found :\n{duplicate_counts}")
else:
# # Proceed with the rest of the code if no duplicates
    print("No duplicates found. Continuing execution...")


No duplicates found. Continuing execution...


In [225]:
column_name = df_output.columns
set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
query = f"""MERGE INTO delta.`{trgt_path_processed}` AS target 
            USING vw_source AS source 
            ON target.PKSK = source.PKSK 
            AND target.RowSK <> source.RowSK 
            WHEN MATCHED THEN UPDATE SET {set_clause}"""
print(query)

MERGE INTO delta.`/mnt/Fact/Fact_Parquet` AS target 
            USING vw_source AS source 
            ON target.PKSK = source.PKSK 
            AND target.RowSK <> source.RowSK 
            WHEN MATCHED THEN UPDATE SET target.SpendingItem = source.SpendingItem, target.SpendAmount = source.SpendAmount, target.WalletSK = source.WalletSK, target.categorysk = source.categorysk, target.DateSK = source.DateSK, target.PKSK = source.PKSK, target.UpdateTimeStamp = source.UpdateTimeStamp, target.RowSK = source.RowSK


In [208]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    spark.sql(query)
else :
    spark.sql(f"CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source")

In [241]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1).write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)

In [242]:
spark.read.format("csv").option("header","true").load(trgt_path_csv).count()

43

In [235]:
spark.sql(f"""SELECT UpdateTimeStamp FROM delta.`{trgt_path_processed}` 
          WHERE UpdateTimeStamp = (SELECT MAX(UpdateTimeStamp) FROM vw_source)""").count()

0